In [1]:
!pip install gradio
!pip install langchain



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.6 MB/s eta 0:00:00


In [2]:
import gradio as gr
from datetime import datetime

def schedule_appointment(patient_name, family_name, start_time, end_time, description, actor, relation, location, reason_for_visit):
    # Validate start and end time
    try:
        start_time = datetime.strptime(start_time, '%Y-%m-%d %H:%M')
        end_time = datetime.strptime(end_time, '%Y-%m-%d %H:%M')
        if end_time <= start_time:
            return "End time must be later than start time."
    except ValueError:
        return start_time

    # Process the appointment and create a confirmation response
    available_slots = ["10:00 AM", "2:00 PM", "4:00 PM"]  # Placeholder for actual calendar check

    # Generate confirmation message
    appointment_details = (
        f"Appointment Details:\n"
        f"Patient Name: {patient_name} {family_name}\n"
        f"Start Time: {start_time.strftime('%Y-%m-%d %H:%M')}\n"
        f"End Time: {end_time.strftime('%Y-%m-%d %H:%M')}\n"
        f"Description: {description}\n"
        f"Actor: {actor}\n"
        f"Relation: {relation}\n"
        f"Location: {location}\n"
        f"Reason for Visit: {reason_for_visit}\n"
        f"Available Slots: {', '.join(available_slots)}"
    )

    return appointment_details


# Create Gradio Interface with additional fields
with gr.Blocks() as demo:
    gr.Markdown("### Miranda's Appointment Scheduling Assistant")

    # Input fields for patient and appointment details
    patient_name_input = gr.Textbox(label="Patient Name", placeholder="Enter patient's first name")
    family_name_input = gr.Textbox(label="Family Member's Name (if applicable)", placeholder="Enter family member's name")
    start_time_input = gr.Textbox(label="Start Time (YYYY-MM-DD HH:MM)", placeholder="e.g., 2025-05-01 3:00")
    end_time_input = gr.Textbox(label="End Time (YYYY-MM-DD HH:MM)", placeholder="e.g., 2025-05-01 4:00")
    description_input = gr.Textbox(label="Description", placeholder="e.g., Routine health check-up appointment")
    actor_input = gr.Dropdown(label="Actor", choices=["Patient", "Practitioner"], value="Patient")
    relation_input = gr.Dropdown(label="Relation", choices=["Self", "Parent"], value="Self")
    location_input = gr.Textbox(label="Location (Physical Address)", placeholder="Enter the address of the appointment")
    reason_input = gr.Textbox(label="Reason for Scheduling", placeholder="e.g., mental health consultation")

    appointment_output = gr.Textbox(label="Appointment Confirmation", interactive=False)
    confirm_button = gr.Button("Confirm Appointment")

    # Define the submit action for the inputs
    confirm_button.click(schedule_appointment,
                         inputs=[patient_name_input, family_name_input, start_time_input, end_time_input,
                                 description_input, actor_input, relation_input, location_input, reason_input],
                         outputs=appointment_output)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://600a4807653f495bc3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [5]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.llms import OpenAI

# Example tool to check calendar availability
def check_calendar_availability(input_reason):
    # Here, we can eventually call Google Calendar API or other backend services
    # For now, we are mocking the availability
    return ["10:00 AM", "2:00 PM", "4:00 PM"]

tools = [
    Tool(name="Check Calendar Availability", func=check_calendar_availability, description="Check available slots.")
]

# Initialize LangChain agent with the tools
llm = OpenAI(temperature=0)
agent = initialize_agent(tools, AgentType.ZERO_SHOT_REACT_DESCRIPTION, llm=llm)

def get_available_slots_from_backend(reason_for_visit):
    # Pass the reason to LangChain and get available slots
    result = agent.run(reason_for_visit)
    return result


ModuleNotFoundError: No module named 'langchain_community'

Next Steps:

Connect Google Calendar API:

The goal is to check availability in both the patient’s and doctor’s calendars. The calendar check function will be passed to LangChain and executed when Miranda requests appointment scheduling.

Here's what we will need to do for the backend integration:

a) Google Calendar API Setup:
We will need to authenticate Google Calendar and set up a function to fetch available slots from both the patient's (Miranda's) calendar and the doctor's calendar.

b) Integrating with LangChain:
We will need to modify the check_calendar_availability() function from above to fetch real-time availability using Google Calendar instead of returning mocked data.


In [6]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import os

SCOPES = ['https://oauth2.googleapis.com/token']

def authenticate_google_calendar():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return creds

def get_calendar_events(calendar_id='primary'):
    creds = authenticate_google_calendar()
    service = build('calendar', 'v3', credentials=creds)
    events_result = service.events().list(calendarId=calendar_id, timeMin='2025-05-01T00:00:00Z',
                                         maxResults=10, singleEvents=True, orderBy='startTime').execute()
    events = events_result.get('items', [])
    return [event['summary'] + " " + event['start']['dateTime'] for event in events]


Coordinate with FHIR Integration:

We need the FHIR API or mock data to pull in appointment records (or any relevant health data) if needed for scheduling or tracking purposes.

Here’s what we need to discuss to integrate FHIR:

FHIR Server Access: Get access to the FHIR endpoint (either a test server or a live server). You'll need to know the FHIR base URL and the required API endpoints for querying patient data, appointment records, and doctor availability.

FHIR Data Integration: For now, we are providing mock data (e.g., a list of appointments or doctor availability) or integrate with a real FHIR server. This data can be passed into the LangChain agent when making scheduling decisions.

In [7]:
import requests
import json

# Function to authenticate and get FHIR data
def authenticate_fhir():
    """
    Authenticate to the FHIR server and get an access token (if necessary)
    This could involve OAuth2 or another method depending on how Josh has set up authentication.
    For this example, we'll assume it's a simple bearer token.
    """
    # Replace with actual FHIR authentication endpoint and credentials
    fhir_auth_url = "https://YOUR_FHIR_SERVER_URL/authenticate"
    headers = {
        "Authorization": "Bearer YOUR_ACCESS_TOKEN"
    }

    # Example to verify connection (optional)
    response = requests.get(fhir_auth_url, headers=headers)
    if response.status_code == 200:
        print("FHIR authentication successful!")
    else:
        print("FHIR authentication failed!", response.text)

# Function to get appointments from the FHIR server
def get_appointments(patient_id):
    """
    Fetch patient appointments from the FHIR server.
    Replace the endpoint with Josh's actual endpoint for getting appointments or related data.
    """
    fhir_url = f"https://YOUR_FHIR_SERVER_URL/Appointment?patient={patient_id}"
    headers = {
        "Authorization": "Bearer YOUR_ACCESS_TOKEN",
        "Accept": "application/fhir+json"
    }

    # Make the request to the FHIR server
    response = requests.get(fhir_url, headers=headers)

    if response.status_code == 200:
        # Assuming the response is in JSON format
        appointments_data = response.json()
        # Process the appointment data
        appointments = []
        for appointment in appointments_data['entry']:
            # Assuming each appointment has a 'resource' with 'start' and 'end' times, etc.
            appointment_info = {
                "doctor": appointment['resource']['participant'][0]['individual']['display'],
                "start_time": appointment['resource']['start'],
                "end_time": appointment['resource']['end'],
                "location": appointment['resource']['location'][0]['display']
            }
            appointments.append(appointment_info)

        return appointments
    else:
        print("Error fetching FHIR data:", response.text)
        return []

# Example function to display available slots using FHIR data
def get_available_fhir_slots(patient_id):
    appointments = get_appointments(patient_id)

    if not appointments:
        return "No appointments found."

    # Example of how to display available slots based on fetched data
    available_slots = []
    for appointment in appointments:
        available_slots.append(f"{appointment['doctor']} - {appointment['start_time']} to {appointment['end_time']} at {appointment['location']}")

    return "\n".join(available_slots)


# Example usage:
patient_id = "PATIENT_ID"  # Replace with actual patient ID, which should be passed or retrieved in the interface
available_slots = get_available_fhir_slots(patient_id)
print(available_slots)  # You can use this data to pass into the LangChain or display on the interface


ConnectionError: HTTPSConnectionPool(host='your_fhir_server_url', port=443): Max retries exceeded with url: /Appointment?patient=PATIENT_ID (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7e4ff5ba1a50>: Failed to resolve 'your_fhir_server_url' ([Errno -2] Name or service not known)"))